# Pandas'ta Join (Birleştirme) İşlemleri Rehberi

Bu rehber, Pandas kütüphanesini kullanarak farklı veri çerçevelerini (DataFrame) birleştirmenin yollarını anlatır. SQL'deki JOIN işlemlerine benzer şekilde, Pandas'ta da `merge` ve `join` fonksiyonları ile tabloları ortak sütunlara veya indekslere göre birleştirebiliriz.

## `pd.merge()` Fonksiyonu

`merge` fonksiyonu, en esnek ve en sık kullanılan birleştirme yöntemidir. Farklı birleştirme türlerini (`inner`, `left`, `right`, `outer`) destekler.

In [ ]:
import pandas as pd

# Örnek Veri Çerçeveleri Oluşturalım

# Çalışanların bilgilerini içeren bir DataFrame
calisanlar = pd.DataFrame({
    'calisan_id': [1, 2, 3, 4],
    'isim': ['Ali', 'Ayşe', 'Veli', 'Fatma'],
    'departman_id': [101, 102, 101, 103]
})

# Departman bilgilerini içeren bir DataFrame
departmanlar = pd.DataFrame({
    'id': [101, 102, 104],
    'departman_adi': ['İnsan Kaynakları', 'Mühendislik', 'Pazarlama']
})

print("--- Çalışanlar Tablosu ---")
print(calisanlar)
print("
--- Departmanlar Tablosu ---")
print(departmanlar)

### 1. Inner Join (İç Birleştirme)

Her iki DataFrame'de de ortak olan anahtarlara (key) göre birleştirme yapar. Eşleşmeyen satırlar sonuçta yer almaz. Bu, `merge` fonksiyonunun varsayılan davranışıdır.

In [ ]:
# 'calisan_id' ve 'id' sütunları üzerinden birleştirme yapıyoruz.
inner_join_df = pd.merge(calisanlar, departmanlar, left_on='departman_id', right_on='id', how='inner')

print("--- Inner Join Sonucu ---")
print(inner_join_df)

*Görüldüğü gibi, Pazarlama departmanı (id: 104) ve Fatma (departman_id: 103) her iki tabloda ortak bir karşılığa sahip olmadıkları için sonuçta yer almadılar.*

### 2. Left Join (Sol Birleştirme)

Sol DataFrame'deki (`calisanlar`) tüm satırları ve sağ DataFrame'den (`departmanlar`) sadece eşleşen satırları alır. Sağ tarafta eşleşme bulunamazsa, o satırın sütunları `NaN` (Not a Number) ile doldurulur.

In [ ]:
left_join_df = pd.merge(calisanlar, departmanlar, left_on='departman_id', right_on='id', how='left')

print("--- Left Join Sonucu ---")
print(left_join_df)

*Fatma'nın departman ID'si (103) departmanlar tablosunda olmadığı için, departman bilgileri `NaN` olarak geldi ama Fatma'nın kendi bilgileri korundu.*

### 3. Right Join (Sağ Birleştirme)

Sağ DataFrame'deki (`departmanlar`) tüm satırları ve sol DataFrame'den (`calisanlar`) sadece eşleşen satırları alır. Sol tarafta eşleşme bulunamazsa, o satırın sütunları `NaN` ile doldurulur.

In [ ]:
right_join_df = pd.merge(calisanlar, departmanlar, left_on='departman_id', right_on='id', how='right')

print("--- Right Join Sonucu ---")
print(right_join_df)

*Pazarlama departmanına (id: 104) atanmış bir çalışan olmadığı için, çalışan bilgileri `NaN` olarak geldi ama departman bilgisi korundu.*

### 4. Outer Join (Dış Birleştirme)

Her iki DataFrame'deki tüm satırları birleştirir. Eşleşme olmayan yerleri `NaN` ile doldurur. Left ve Right Join'in birleşimi gibidir.

In [ ]:
outer_join_df = pd.merge(calisanlar, departmanlar, left_on='departman_id', right_on='id', how='outer')

print("--- Outer Join Sonucu ---")
print(outer_join_df)

*Hem departmanı olmayan çalışan (Fatma) hem de çalışanı olmayan departman (Pazarlama) sonuç tablosunda yer aldı.*

## `df.join()` Metodu

`join` metodu, DataFrame'leri indekslerine veya belirtilen bir sütuna göre birleştirmek için daha basit bir arayüz sunar. `pd.merge()`'e göre daha az esnek olsa da, özellikle indeks tabanlı birleştirmeler için oldukça kullanışlıdır. Varsayılan olarak sol birleştirme (`how='left'`) yapar.

In [ ]:
# join metodu için indeksleri ayarlayalım
calisanlar_indexed = calisanlar.set_index('departman_id')
departmanlar_indexed = departmanlar.set_index('id')

print("--- İndekslenmiş Çalışanlar ---")
print(calisanlar_indexed)
print("
--- İndekslenmiş Departmanlar ---")
print(departmanlar_indexed)

# İndeks üzerinden join işlemi (varsayılan 'left' join)
join_df = calisanlar_indexed.join(departmanlar_indexed)

print("
--- df.join() Sonucu ---")
print(join_df)

*Görüldüğü gibi, `calisanlar_indexed` DataFrame'inin indeksi (`departman_id`) ile `departmanlar_indexed` DataFrame'inin indeksi (`id`) üzerinden bir birleştirme yapıldı. `merge`'deki `left join` ile aynı sonucu verdi ve Fatma'nın departman bilgisi `NaN` olarak geldi.*

## `merge` vs. `join`: Ne Zaman Hangisini Kullanmalı?

Her iki fonksiyon da benzer amaçlara hizmet etse de aralarında bazı temel farklar vardır:

- **Kullanım Şekli:** `merge` bir Pandas fonksiyonudur (`pd.merge(df1, df2)`), `join` ise bir DataFrame metodudur (`df1.join(df2)`).
- **Birleştirme Anahtarı:** `merge` varsayılan olarak ortak sütun adlarına göre birleştirme yapar, ancak `left_on` ve `right_on` ile farklı sütun adları belirtebilirsiniz. `join` ise varsayılan olarak DataFrame'lerin indekslerine göre birleştirme yapar.
- **Esneklik:** `merge`, SQL'deki JOIN mantığına çok benzer ve çok daha fazla esneklik sunar. Farklı türde birleştirmeler (`inner`, `left`, `right`, `outer`) ve karmaşık senaryolar için daha uygundur.

**Özetle:**
- Eğer SQL'e benzer şekilde, sütunlar üzerinden esnek birleştirme yapmak istiyorsanız `pd.merge()`'ü tercih edin.
- Eğer iki DataFrame'i hızlıca indeksleri üzerinden birleştirmek istiyorsanız `df.join()` daha pratik bir seçenektir.